# 4.2 - Office finder 

#### Encontramos tu oficina

In [2]:
%pip install folium

  Using cached folium-0.12.1.post1-py2.py3-none-any.whl (95 kB)
  Using cached branca-0.4.2-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pymongo import MongoClient

import pandas as pd

import re

import folium

In [4]:
cliente=MongoClient('mongodb://localhost:27017')

In [6]:
db=cliente.companies

In [13]:
def data():

    query=db.companies.find({'$and': [
                                        {'$or': [
                                            {'category_code': 'games_video'},
                                            {'category_code': 'software'},
                                            {'category_code': 'web'},
                                            {'category_code': 'mobile'},
                                            {'category_code': 'photo_video'},
                                            {'category_code': 'ecommerce'},
                                            {'category_code': 'search'},
                                            {'category_code': 'network_hosting'}]}, # aqui se cierra el or

                                    {'founded_year': {'$gte': 1999}},
                                    {'offices': {'$ne': []}},  
                                    {'offices.latitude': {'$ne': []}},
                                    {'offices.longitude': {'$ne': []}},
                                    {'offices.latitude': {'$ne': None}},
                                    {'offices.longitude': {'$ne': None}},

                                    {'deadpooled_year': None},
                                    {'number_of_employees': {'$ne': None}},
                                    {'total_money_raised': {'$ne': None}}]},  # aqui se cierra el and


                           {'name': 1, '_id': 0, 'offices.latitude':1, 'offices.longitude': 1,
                           'founded_year': 1, 'category_code': 1, 'number_of_employees': 1,
                           'total_money_raised': 1}   # selccion de keys que quiero
                           )



    df=pd.json_normalize(data=query, record_path='offices', 
                         meta=['category_code', 'name', 'number_of_employees',
                                'founded_year', 'total_money_raised'])

    df.insert(2, 'geo_loc', pd.Series(list(range(len(df)))))

    df.geo_loc=df.apply(lambda x: [x['longitude'], x['latitude']], axis=1)

    return df

In [15]:
data().tail(10)

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised
2871,29.735462,-95.413243,"[-95.413243, 29.735462]",games_video,Fictionaut,2,2008,$0
2872,40.756054,-73.986951,"[-73.986951, 40.756054]",web,Conversition,10,2008,$0
2873,48.869995,2.340695,"[2.3406954, 48.8699947]",software,PubliAtis,10,2006,€1M
2874,59.617364,16.559539,"[16.5595393, 59.617364]",web,Gastrogate,12,1999,$0
2875,49.282455,-123.109217,"[-123.109217, 49.2824551]",games_video,Tiny Speck,45,2009,$17.2M
2876,38.752367,-9.146221,"[-9.1462209, 38.7523672]",games_video,My TV Shows,2,2007,$0
2877,54.323738,10.166515,"[10.1665152, 54.323738]",network_hosting,AwardSpace,35,2004,$0
2878,43.676252,-79.400859,"[-79.4008589, 43.6762521]",search,Buddhacom,8,2008,$0
2879,51.517742,-0.116537,"[-0.1165365, 51.5177425]",mobile,acrossair,6,2009,$0
2880,50.839970,4.346472,"[4.3464721, 50.83997]",mobile,Getyoo,10,2009,€1.13M


In [27]:
def exchange(money):  # entra una string
    
    res=[]
    
    number=1
    symbol=1
    
    for e in money:
        
        if e[-1]=='k': number=1e3
        elif e[-1]=='M': number=1e6
        elif e[-1]=='B': number=1e9
            
            
        if e[0]=='£': symbol=1.3
        elif e[0]=='€': symbol=1.12

        n_money=re.findall('[0-9.]+', e)[0]

        res.append(int(float(n_money) * number * symbol))
        
    return res

In [28]:
exchange(['€1.13M'])

[1265600]

In [29]:
df=data()

df['n_total_money']=exchange(df.total_money_raised)

df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000


In [32]:
def classifying(year, n_employees):
    
    clase=[]
    
    for i in range(len(year)):
        
        if year[i]>=2008 and n_employees[i]<100:
            
            clase.append('StartUp')
            
        else:
            
            clase.append('BigCorp')
            
    return clase
        

In [36]:
df['company_class']=classifying(df.founded_year, df.number_of_employees)

df.tail()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class
2876,38.752367,-9.146221,"[-9.1462209, 38.7523672]",games_video,My TV Shows,2,2007,$0,0,BigCorp
2877,54.323738,10.166515,"[10.1665152, 54.323738]",network_hosting,AwardSpace,35,2004,$0,0,BigCorp
2878,43.676252,-79.400859,"[-79.4008589, 43.6762521]",search,Buddhacom,8,2008,$0,0,StartUp
2879,51.517742,-0.116537,"[-0.1165365, 51.5177425]",mobile,acrossair,6,2009,$0,0,StartUp
2880,50.839970,4.346472,"[4.3464721, 50.83997]",mobile,Getyoo,10,2009,€1.13M,1265600,StartUp


In [37]:
df.shape  # estoy considerando 2881 empresas

(2881, 10)

In [38]:
db.geoloc.insert_many(df.to_dict('records'))  # creada la coleccion

In [39]:
# crear el indice 2d

db.geoloc.create_index([('geo_loc', '2dsphere')])

'geo_loc_2dsphere'

In [47]:
# geolocalizacion


def geoquery(df):
    
    
    list_money_mean=[]
    
    sb_ratio=[]
    
    n_companies=[]
    
    for i in range(len(df)):
        
        # lista de diccionarios
        query=list(db.geoloc.find({
            'geo_loc': {'$near': {'$geometry': {'type': 'Point',
                                                 'coordinates': [df.longitude[i], df.latitude[i]]},
                                  '$maxDistance': 1000}}
        }))
        
        long=len(query)  # numero de compañias que tiene alrededor
        
        n_companies.append(long)
        
        money_mean=sum([query[j]['n_total_money'] for j in range(long)])/long  # media de dinero
        
        list_money_mean.append(money_mean)
        
        big=sum([query[k]['company_class'].count('BigCorp') if query[k]['company_class'].count('BigCorp')!=0 else 1 for k in range(long)])
        start=sum([query[k]['company_class'].count('StartUp') for k in range(long)])
        
        sb_ratio.append(start/big)
        
    df['around_companies']=n_companies
    
    df['money_density']=list_money_mean
    
    df['sb_ratio']=sb_ratio
    
    return df


In [48]:
df=geoquery(df)

df.head()

,latitude,longitude,geo_loc,category_code,name,number_of_employees,founded_year,total_money_raised,n_total_money,company_class,around_companies,money_density,sb_ratio
0,34.090368,-118.393064,"[-118.393064, 34.090368]",web,Geni,18,2006,$16.5M,16500000,BigCorp,1,1.650000e+07,0.000000
1,40.746497,-74.009447,"[-74.0094471, 40.7464969]",games_video,Joost,0,2006,$45M,45000000,BigCorp,8,7.325250e+06,0.375000
2,37.387845,-122.055197,"[-122.055197, 37.387845]",web,Plaxo,50,2002,$28.3M,28300000,BigCorp,2,2.715000e+07,0.000000
3,37.778613,-122.395289,"[-122.395289, 37.778613]",search,Powerset,60,2006,$22.5M,22500000,BigCorp,27,1.105724e+07,0.222222
4,34.017606,-118.487267,"[-118.487267, 34.017606]",web,Mahalo,40,2007,$21M,21000000,BigCorp,3,8.733333e+06,0.333333


In [49]:
def geomap(df, zoom_start=1.5):
    
    
    df.index=range(len(df))
    
    mapa=folium.Map(location=[0, 0],
                    titles='openstreetmap', zoom_start=zoom_start)
    
    
    for i in range(1, len(df)):
        
        folium.CircleMarker([float(df.latitude[i]), float(df.longitude[i])],
                            popup=str(df.name[i]),
                            radius=1, icon=folium.Icon()).add_to(mapa)
        
    mapa.save('images/geomap.html')
    
    
    return mapa

In [50]:
df.describe()

,latitude,longitude,n_total_money,around_companies,money_density,sb_ratio
count,2881.000000,2881.000000,2.881000e+03,2881.000000,2.881000e+03,2881.000000
mean,38.052028,-60.793188,4.921155e+06,6.052412,4.908734e+06,0.281803
std,15.876878,66.289707,2.141485e+07,11.280223,1.455534e+07,0.365205
min,-41.565586,-159.480262,0.000000e+00,1.000000,0.000000e+00,0.000000
25%,35.911733,-118.382327,0.000000e+00,1.000000,0.000000e+00,0.000000
50%,40.016582,-79.420926,0.000000e+00,2.000000,0.000000e+00,0.100000
75%,45.629498,-0.427418,1.120000e+05,5.000000,4.395730e+06,0.500000
max,65.056601,175.260400,4.090000e+08,75.000000,2.526000e+08,1.000000


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2881 entries, 0 to 2880
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   latitude             2881 non-null   float64
 1   longitude            2881 non-null   float64
 2   geo_loc              2881 non-null   object 
 3   category_code        2881 non-null   object 
 4   name                 2881 non-null   object 
 5   number_of_employees  2881 non-null   object 
 6   founded_year         2881 non-null   object 
 7   total_money_raised   2881 non-null   object 
 8   n_total_money        2881 non-null   int64  
 9   company_class        2881 non-null   object 
 10  around_companies     2881 non-null   int64  
 11  money_density        2881 non-null   float64
 12  sb_ratio             2881 non-null   float64
dtypes: float64(4), int64(2), object(7)
memory usage: 292.7+ KB


In [52]:
df.shape

(2881, 13)

In [53]:
geomap(df.loc[:1000])

In [55]:
print('Compañias alrededor=10, sb_ratio>0.45 y densidad de pasta>1.000.000$')

data=df[(df.around_companies>=10) & (df.sb_ratio>0.45) & (df.money_density>1e6)]

geomap(data)

Compañias alrededor=10, sb_ratio>0.45 y densidad de pasta>1.000.000$


In [56]:
print('Compañias alrededor>=70, sb_ratio>0.25 y densidad de pasta>10.000.000$')

data=df[(df.around_companies>=70) & (df.sb_ratio>0.25) & (df.money_density>1e7)]

geomap(data)

Compañias alrededor>=70, sb_ratio>0.25 y densidad de pasta>10.000.000$
